# Using `lyra-fr` on SageMaker through Model Packages

Developed by <a href="https://lighton.ai/"/>LightOn</a>, `lyra-fr` is the largest language model natively trained in French. `lyra-fr` understands natural language instructions in French, and can perform many tasks in zero-shot or few-shot. If you want to know more about the best ways to prompt large language models, you can have a look at the <a href="https://muse-docs.lighton.ai/usecases/english/prompt">documentation</a>. If you are familiar with the prompting literature, advanced techniques like <a href="https://arxiv.org/abs/2201.11903">Chain of Thought</a> also work with it.

Summarizing is as easy as adding *Résumé :* after the relevant text snippet, simply change it to *Mots clés :* to perform keywords extraction instead. The only limit is what you can express in text.

For example

>Extrais les mots clés de l'article suivant: Le corium est un magma métallique et minéral constitué d'éléments fondus du cœur d'un réacteur nucléaire, puis des minéraux qu'il peut absorber lors de son trajet. Le terme « corium » est un néologisme formé de core (en anglais, pour le cœur d'un réacteur nucléaire), suivi du suffixe -ium présent dans le nom de nombreux éléments du tableau périodique des éléments : lithium, calcium, uranium, plutonium, hélium, strontium, etc. Initialement constitué du combustible nucléaire (principalement de l'oxyde d'uranium enrichi), des éléments de l'assemblage combustible et des divers équipements du cœur (barres de contrôle, instrumentation) ou de la paroi de la cuve du réacteur avec lesquels il entre en contact, il se forme à très haute température (environ 3 000 °C, température de fusion de l'oxyde d'uranium) quand le cœur n'est plus refroidi, comme lors d'accidents nucléaires tels ceux de Three Mile Island, de Tchernobyl, ou de Fukushima.
>
>Mots-clés : corium, réacteur nucléaire, fusion du cœur

This sample notebook shows you how to deploy `lyra-fr` using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.

## Contents:
1. [Select model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select to the model package
Confirm that you received this notebook from model catalog on SageMaker AWS Management Console.

In [1]:
# Mapping for Model Packages (initially only us-east-1 and eu-west-1 is supported)
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/lyra-fr-13be5bdcc7b73aca97c39905991719ed",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/lyra-fr-13be5bdcc7b73aca97c39905991719ed",
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

In [3]:
%pip install git+https://github.com/lightonai/lightonmuse-sagemaker-sdk.git

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Cloning https://github.com/lightonai/lightonmuse-sagemaker-sdk.git to /tmp/pip-req-build-hcjrrdma
  Running command git clone --filter=blob:none --quiet https://github.com/lightonai/lightonmuse-sagemaker-sdk.git /tmp/pip-req-build-hcjrrdma
  Resolved https://github.com/lightonai/lightonmuse-sagemaker-sdk.git to commit 97ba0085b56bf988b58962285600f78a4c6830ba
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import lightonsage as lsage

In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [6]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [7]:
model_name = "lyra-fr"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.p4d.24xlarge"
)

### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, model_data_download_timeout=3600)

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:372939818206:endpoint-config/lyra-fr".

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

For more details on the parameters of the endpoint Create, see the <a href="https://muse-docs.lighton.ai/api/endpoints/create">docs</a>.

In [9]:
payload = {
    "text": "Génère une liste d'idées d'articles sur l'aquarelle.\n1. L'aquarelle dans l'histoire\n2.", 
    "params": {
        "n_tokens": 43, 
        "seed": 0
    }, 
    "endpoint":"create"
}

<Add code snippet that shows the payload contents>

### C. Perform real-time inference

In [10]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=model_name,
    ContentType="application/json",
    Body=json.dumps(payload),
 
)

output = json.loads(response["Body"].read().decode("utf8"))

### D. Visualize output

In [11]:
print(f"{payload['text']} 🤖 {output['outputs'][0][0]['completions'][0]['output_text']}")

Génère une liste d'idées d'articles sur l'aquarelle.
1. L'aquarelle dans l'histoire
2. 🤖  L'histoire de l'aquarelle
3. Le matériel de l'aquarelliste
4. Les pigments utilisés en aquarelle
5. Les encres en aquarelle



You have seen above how to use JSON payloads to invoke an endpoint, now let's use the Python SDK. Using the Python SDK provides a better, more streamlined user experience, and it is recommeneded for experimentation.

The Python SDK takes care for you of formatting the input, calling the endpoint, and unpacking the output. There is one class per endpoint: `Create`, `Analyse`, `Select`, `Embed`, `Compare` and `Tokenize`.

#### Using Create for sentiment analysis

You should describe the task you want to carry out in French, for example in the following: *Détermine si ces commentaires expriment des avis positifs, négatifs ou mitigés.*

In addition, providing examples improves the performance of the model.

In [12]:
creator = lsage.Create("lyra-fr")

In [13]:
few_shot_prompt = examples = f"""Détermine si ces commentaires expriment des avis positifs, négatifs ou mitigés.
----------
Commentaire : \"Désolé mais je n'ai pas de compliment à faire à ce garage. Je me trouvais sur Saint-Étienne j'étais crevé avec bien sur un clou dans le pneu, l employé me dit qu il ne pouvais pas me dépanner, qu'il fallait que je gonfle mon pneu à 2.4 je suis redescendu à Roanne il a fallu que je arrête mi-chemin pour gonfler mon pneu. Chercher l'erreur\"
Ce commentaire exprime un avis négatif.
----------
Commentaire : \"Garage avec un service bien meilleur que les concurrents.Ça aurait pu être parfait, si un léger coup de nettoyage sur les jantes lors du remplacement de mes pneus avait été effectué.Mes jantes était pour le coup plus sales qu'à l'arrivée à cause du liquide utilisé pour le montage des pneus qui n'a pas été essuyé.Mais après tout ça n'est qu'un détail bonne expérience en général.Je commande !\"
Ce commentaire exprime un avis positif.
----------
Commentaire : \"C'est correct mais des employés pas forcément tous très bien qualifiés\"
Ce commentaire exprime un avis mitigé.
----------
Commentaire : \"Garantie de batterie contraignante et compliquée, du marketing abusif. Si elle est défaillante, on vous la change, mais on vous fait payer la main d'œuvre de remplacement !\"
Ce commentaire exprime un avis négatif.
----------
Commentaire : \"Bon accueil, heure de rendez-vous respecté et le prix de la révision bien moins cher que dans un garage de marque de voiture, prochaine révision ou n’importe quel autre service je le ferai dans chez Norauto, très bon garage.\"
Ce commentaire exprime un avis positif.
----------
Commentaire: \"Excellente enseigne. Personnel compétent et agréable. Je mets cette adresse en favori.\"
Ce commentaire exprime un avis"""
outputs, costs, request_id = creator(few_shot_prompt, stop_words=[".", "\n"])
print(f"{few_shot_prompt} 🤖 {outputs[0]['completions'][0]['output_text']}")

Détermine si ces commentaires expriment des avis positifs, négatifs ou mitigés.
----------
Commentaire : "Désolé mais je n'ai pas de compliment à faire à ce garage. Je me trouvais sur Saint-Étienne j'étais crevé avec bien sur un clou dans le pneu, l employé me dit qu il ne pouvais pas me dépanner, qu'il fallait que je gonfle mon pneu à 2.4 je suis redescendu à Roanne il a fallu que je arrête mi-chemin pour gonfler mon pneu. Chercher l'erreur"
Ce commentaire exprime un avis négatif.
----------
Commentaire : "Garage avec un service bien meilleur que les concurrents.Ça aurait pu être parfait, si un léger coup de nettoyage sur les jantes lors du remplacement de mes pneus avait été effectué.Mes jantes était pour le coup plus sales qu'à l'arrivée à cause du liquide utilisé pour le montage des pneus qui n'a pas été essuyé.Mais après tout ça n'est qu'un détail bonne expérience en général.Je commande !"
Ce commentaire exprime un avis positif.
----------
Commentaire : "C'est correct mais des emp

#### Using Select for review classification

In [14]:
selector = lsage.Select("lyra-fr")

In [23]:
prompt = """Étant donné un commentaire et la catégorie à laquelle il appartient, détermine la sous-catégorie la plus appropriée.

Commentaire : \"Venu pour faire réparer ma roue crevée, la conseillière a rapidement constater que le pneu n'est pas réparable. Elle m'a proposé des pneus équivalents et pris rdv pour l'après-midi, car un samedi matin, le centre était saturé. Lors de l'arrivée au rdv, prise en charge rapide et 30 min plus tard, le véhicule était prêt - Avertissement par SMS. Même lors du paiement, l'hôte de caisse aura été agréable.Excellent accueil et super boulot\"
Catégorie :"""

In [26]:
outputs, costs, request_id = selector(reference=prompt, candidates=["accueil et conseils", "prise rendez-vous"])
print(f"{prompt} 🤖 {outputs[0]['best']}")

Étant donné un commentaire et la catégorie à laquelle il appartient, détermine la sous-catégorie la plus appropriée.

Commentaire : "Venu pour faire réparer ma roue crevée, la conseillière a rapidement constater que le pneu n'est pas réparable. Elle m'a proposé des pneus équivalents et pris rdv pour l'après-midi, car un samedi matin, le centre était saturé. Lors de l'arrivée au rdv, prise en charge rapide et 30 min plus tard, le véhicule était prêt - Avertissement par SMS. Même lors du paiement, l'hôte de caisse aura été agréable.Excellent accueil et super boulot"
Catégorie : 🤖 accueil et conseils


### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [27]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [28]:
model.delete_model()